In [36]:
import json
import warnings
import requests
import tqdm

In [50]:
#Load your api key
with open('api_keys.json') as f:
    api_keys = json.load(f)
alt_metric_key = api_keys['alt_metric']
def get_reduced_alt_values(r):
    """when some info is missing extracts less information from the response object"""
    total_readers = 0
    for key in r['counts']['readers'].keys():
        total_readers += int(r['counts']['readers'][key])
    score = r['score']
    percentile = -100 #r['altmetric_score']['context_for_score']['all']['percentile']
    similar_age_3m_percentile= -100 #r['altmetric_score']['context_for_score']['similar_age_3m']  #['percentile']
    alt_values = {'score': score}, {'percentile': percentile}, {'similar_age_3m_percentile': similar_age_3m_percentile}, {'total_readers': total_readers}
    return alt_values
def get_alt_values(r):
    """extracts relevant information from the response object"""
    total_readers = 0
    for key in r['counts']['readers'].keys():
        total_readers += int(r['counts']['readers'][key])
    score = r['score']
    percentile = 0 #r['altmetric_score']['context_for_score']['all']['percentile']
    similar_age_3m_percentile= r['altmetric_score']['context_for_score']['similar_age_3m']  #['percentile']
    alt_values = {'score': score}, {'percentile': percentile}, {'similar_age_3m_percentile': similar_age_3m_percentile}, {'total_readers': total_readers}
    return alt_values
def altmetric_arxiv_requests(arxiv):
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/arxiv_id/{arxiv}?key={alt_metric_key}').json()
    return r
def altmetric_doi_requests(doi):
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/doi/{doi}?key={alt_metric_key}').json()
    return r
altmetric_doi_requests('10.1177%2F0278364913491297')['altmetric_score']['context_for_score']['all']

{'total_number_of_other_articles': 22545203,
 'mean': 10.140114810243793,
 'rank': 2379437,
 'this_scored_higher_than_pct': 89,
 'this_scored_higher_than': 20154515,
 'rank_type': 'exact',
 'sample_size': 22545203,
 'percentile': 89}

In [52]:
suffix = '_22_12_2022'
file_name = f'data/data_sorted_only_w_ids{suffix}.json'
with open(file_name, "r") as ds:
    retrieved_data = json.load(ds)
print('Number of Datasets:', len(retrieved_data))
c = 0
c_doi = 0
for idx_paper, paper in enumerate(retrieved_data):
    c += 1
    try:
        doi = paper['DOI']
        r = altmetric_doi_requests(doi)
        c_doi += 1
    except:
        try:
            r = altmetric_arxiv_requests(paper['arxivId'])
            c_doi += 1
        except Exception as ex:
            warnings.warn(f'{paper["id"]} caused: {ex}')
        continue
    try:
        alt_values = get_alt_values(r)
    except:
        try:
            alt_values = get_reduced_alt_values(r)
        except Exception as exec:
            print(exec)
            c_doi -= 1
    retrieved_data[idx_paper].update({'altmetrics': alt_values})
with open(f'data/data_sorted_only_w_ids{suffix}_altmetrics.json', 'w', encoding='utf-8') as f:
    json.dump(retrieved_data, f)
print('found:', c_doi, '/', c)

Number of Datasets: 186


C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: GTSDB caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: Malaga Stereo and Laser Urban caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: Stanford Track Collection caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: Complex Urban caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: UAH-DriveSet caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\Temp\ipykernel_16296\1662549837.py:19: UserWarning: HD1K caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')
C:\Users\jonas\AppData\Local\T

found: 159 / 186


In [53]:
zero_counter = 0
for data in retrieved_data:
    try:
        perc = data['altmetrics']['similar_age_3m_percentile']
        if perc == 0:
            zero_counter += 1
    except:
        continue
zero_counter

0